## 구글 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content


In [ ]:
!git clone https://github.com/ultralytics/ultralytics
%cd ultralytics

Cloning into 'ultralytics'...
remote: Enumerating objects: 17085, done.
remote: Total 17085 (delta 0), reused 0 (delta 0), pack-reused 17085
Receiving objects: 100% (17085/17085), 8.98 MiB | 14.87 MiB/s, done.
Resolving deltas: 100% (11939/11939), done.
/content/ultralytics


In [ ]:
!ls -al

total 192
drwxr-xr-x  9 root root  4096 Oct 26 04:07 .
drwxr-xr-x  1 root root  4096 Oct 26 04:07 ..
-rw-r--r--  1 root root   612 Oct 26 04:07 CITATION.cff
-rw-r--r--  1 root root  5597 Oct 26 04:07 CONTRIBUTING.md
drwxr-xr-x  2 root root  4096 Oct 26 04:07 docker
drwxr-xr-x 17 root root  4096 Oct 26 04:07 docs
drwxr-xr-x  8 root root  4096 Oct 26 04:07 examples
drwxr-xr-x  8 root root  4096 Oct 26 04:07 .git
drwxr-xr-x  4 root root  4096 Oct 26 04:07 .github
-rw-r--r--  1 root root  2295 Oct 26 04:07 .gitignore
-rw-r--r--  1 root root 34523 Oct 26 04:07 LICENSE
-rw-r--r--  1 root root   200 Oct 26 04:07 MANIFEST.in
-rw-r--r--  1 root root 24283 Oct 26 04:07 mkdocs.yml
-rw-r--r--  1 root root  2192 Oct 26 04:07 .pre-commit-config.yaml
-rw-r--r--  1 root root 28991 Oct 26 04:07 README.md
-rw-r--r--  1 root root 27890 Oct 26 04:07 README.zh-CN.md
-rw-r--r--  1 root root  1244 Oct 26 04:07 requirements.txt
-rw-r--r--  1 root root  2016 Oct 26 04:07 setup.cfg
-rw-r--r--  1 root root  4004

In [ ]:
dataPath ='/content/drive/MyDrive/Proj/고속도로CCTV데이터기반차량인식/data/bbox_highway/'

## To convert MS COCO xml to YOLO V5 annotation files

In [ ]:
import numpy as np
import lxml
import os

from lxml import etree

CLASSES = ["car", "bus", "truck"]

def to_yolov8(y):
  """
  # change to yolo v8 format
  # [x_top_left, y_top_left, x_bottom_right, y_bottom_right] to
  # [x_center, y_center, width, height]
  """
  width = y[2] - y[0]
  height = y[3] - y[1]

  if width < 0 or height < 0:
      print("ERROR: negative width or height ", width, height, y)
      raise AssertionError("Negative width or height")
  return (y[0] + (width/2)), (y[1] + (height/2)), width, height

def load_xml_annotations(f):
  tree = etree.parse(f)
  anns = []
  for dim in tree.xpath("image"):
    image_filename = dim.attrib["name"]
    width = int(dim.attrib["width"])
    height = int(dim.attrib["height"])
    # print(image_filename)
    # print(len(dim.xpath("box")))
    boxes = []
    for box in dim.xpath("box"):
      label = CLASSES.index(box.attrib["label"])
      xtl, ytl = box.attrib["xtl"], box.attrib["ytl"]
      xbr, ybr = box.attrib["xbr"], box.attrib["ybr"]

      xc, yc, w, h = to_yolov8([float(xtl), float(ytl), float(xbr), float(ybr)])
      boxes.append([label, round(xc/width, 5), round(yc/height, 5), round(w/width, 5), round(h/height, 5)])

    anns.append([image_filename, width, height, boxes])

  return np.array(anns)

label_file = os.path.join(dataPath,'Suwon_CH02_20200722_1600_WED_9m_RH_highway_TW5_rainy_FHD.xml')

anns = load_xml_annotations(label_file)

<ipython-input-8-70f5cc17e186>:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(anns)


In [ ]:
import os
from glob import glob

trainPath = os.path.join(dataPath,'train')
validPath = os.path.join(dataPath,'val')
testPath  = os.path.join(dataPath,'test')

trainImagesPath = os.path.join(trainPath, 'images')
validImagesPath = os.path.join(validPath, 'images')
testImagesPath  = os.path.join(testPath,  'images')

trainLabelsPath = os.path.join(trainPath, 'labels')
validLabelsPath = os.path.join(validPath, 'labels')
testLabelsPath  = os.path.join(testPath,  'labels')

In [ ]:
os.makedirs(trainImagesPath, exist_ok=True)
os.makedirs(validImagesPath, exist_ok=True)
os.makedirs(testImagesPath, exist_ok=True)

os.makedirs(trainLabelsPath, exist_ok=True)
os.makedirs(validLabelsPath, exist_ok=True)
os.makedirs(testLabelsPath, exist_ok=True)

In [ ]:
trainImagesPath

'/content/drive/MyDrive/Proj/고속도로CCTV데이터기반차량인식/data/bbox_highway/train/images'

In [ ]:
# for filename in glob(os.path.join(trainPath,'*.png')):
#     basename = os.path.basename(filename)
#     os.rename(filename, os.path.join(trainImagesPath,basename))

# for filename in glob(os.path.join(validPath,'*.png')):
#     basename = os.path.basename(filename)
#     os.rename(filename, os.path.join(validImagesPath,basename))

# for filename in glob(os.path.join(testPath,'*.png')):
#     basename = os.path.basename(filename)
#     os.rename(filename, os.path.join(testImagesPath,basename))


# for filename in glob(os.path.join(trainPath,'*.txt')):
#     basename = os.path.basename(filename)
#     os.rename(filename, os.path.join(trainLabelsPath,basename))

# for filename in glob(os.path.join(validPath,'*.txt')):
#     basename = os.path.basename(filename)
#     os.rename(filename, os.path.join(validLabelsPath,basename))

# for filename in glob(os.path.join(testPath,'*.txt')):
#     basename = os.path.basename(filename)
#     os.rename(filename, os.path.join(testLabelsPath,basename))

In [ ]:
import torch

In [ ]:
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} \
    ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 2.1.0+cu118     (Tesla V100-SXM2-16GB)


# Train Vehicle Detection Model

In [ ]:
%cd ultralytics
!ls -al

/content/ultralytics/ultralytics
total 48
drwxr-xr-x 11 root root 4096 Oct 26 04:07 .
drwxr-xr-x  9 root root 4096 Oct 26 04:07 ..
drwxr-xr-x  2 root root 4096 Oct 26 04:07 assets
drwxr-xr-x  5 root root 4096 Oct 26 04:07 cfg
drwxr-xr-x  3 root root 4096 Oct 26 04:07 data
drwxr-xr-x  2 root root 4096 Oct 26 04:07 engine
drwxr-xr-x  2 root root 4096 Oct 26 04:07 hub
-rw-r--r--  1 root root  463 Oct 26 04:07 __init__.py
drwxr-xr-x  8 root root 4096 Oct 26 04:07 models
drwxr-xr-x  3 root root 4096 Oct 26 04:07 nn
drwxr-xr-x  3 root root 4096 Oct 26 04:07 trackers
drwxr-xr-x  3 root root 4096 Oct 26 04:07 utils


In [ ]:
from IPython.core.magic import register_line_cell_magic

# yaml에 직접 쓰기를 실행하는 함수
@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/drive/MyDrive/Proj/고속도로CCTV데이터기반차량인식/data/bbox_highway/data.yaml
train: /content/drive/MyDrive/Proj/고속도로CCTV데이터기반차량인식/data/bbox_highway/train/images
val: /content/drive/MyDrive/Proj/고속도로CCTV데이터기반차량인식/data/bbox_highway/val/images
test: /content/drive/MyDrive/Proj/고속도로CCTV데이터기반차량인식/data/bbox_highway/test/images

nc: 3
names: ['car', 'bus', 'truck']

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.yaml')  # build a new model from YAML
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)
model = YOLO('yolov8n.yaml').load('yolov8n.pt')  # build from YAML and transfer weights

# Train the model
yamlPath = '/content/drive/MyDrive/Proj/고속도로CCTV데이터기반차량인식/data/bbox_highway/data.yaml'
results = model.train(data=yamlPath, epochs=100, imgsz=640)


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8s.yaml')  # build a new model from YAML
model = YOLO('yolov8s.pt')  # load a pretrained model (recommended for training)
model = YOLO('yolov8s.yaml').load('yolov8s.pt')  # build from YAML and transfer weights

# Train the model
yamlPath = '/content/drive/MyDrive/Proj/고속도로CCTV데이터기반차량인식/data/bbox_highway/data.yaml'
results = model.train(data=yamlPath, epochs=400, imgsz=640, batch=10)


                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256

In [ ]:
import os
os.getcwd()

'/content/ultralytics/ultralytics'

In [ ]:
testImages = '/content/drive/MyDrive/Proj/고속도로CCTV데이터기반차량인식/data/bbox_highway/test/images'
from glob import glob

testList =[]
for filename in glob(os.path.join(testImages, '*.png')):
    testList.append(filename)

In [ ]:
testList

['/content/drive/MyDrive/Proj/고속도로CCTV데이터기반차량인식/data/bbox_highway/test/images/Suwon_CH02_20200722_1600_WED_9m_RH_highway_TW5_rainy_FHD_099.png',
 '/content/drive/MyDrive/Proj/고속도로CCTV데이터기반차량인식/data/bbox_highway/test/images/Suwon_CH02_20200722_1600_WED_9m_RH_highway_TW5_rainy_FHD_097.png',
 '/content/drive/MyDrive/Proj/고속도로CCTV데이터기반차량인식/data/bbox_highway/test/images/Suwon_CH02_20200722_1600_WED_9m_RH_highway_TW5_rainy_FHD_100.png',
 '/content/drive/MyDrive/Proj/고속도로CCTV데이터기반차량인식/data/bbox_highway/test/images/Suwon_CH02_20200722_1600_WED_9m_RH_highway_TW5_rainy_FHD_098.png',
 '/content/drive/MyDrive/Proj/고속도로CCTV데이터기반차량인식/data/bbox_highway/test/images/Suwon_CH02_20200722_1600_WED_9m_RH_highway_TW5_rainy_FHD_096.png']

In [ ]:
from ultralytics import YOLO

resultList =[]

# Load a pretrained YOLOv8n model
model = YOLO('/content/ultralytics/ultralytics/runs/detect/train2/weights/best.pt')

# Run inference on images
results = model(testList)

# View results
for result in results:
    resultList.append(result.cpu().numpy().boxes.cls)  # print the Boxes object containing the detection bounding boxes


0: 384x640 55 cars, 4 buss, 20 trucks, 1: 384x640 8 cars, 9 trucks, 2: 384x640 37 cars, 2 buss, 23 trucks, 3: 384x640 42 cars, 1 bus, 31 trucks, 4: 384x640 13 cars, 4 trucks, 11.4ms
Speed: 2.1ms preprocess, 2.3ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)


In [ ]:
import pandas as pd
sum = [0, 0, 0]

for result in resultList:
    for i in range(len(sum)):
        sum[i] += result.tolist().count(i)
print(sum)

print("전체 {}장의 이미지에서 승용차 {}대, 버스 {}대, 트럭 {}대를 발견하였습니다.".format(len(resultList), sum[0], sum[1], sum[2]))

[155, 7, 87]
전체 5장의 이미지에서 승용차 155대, 버스 7대, 트럭 87대를 발견하였습니다.
